# Application of Radial Equilibrium Equation
## Adaptation of the code RE-DES by Lewis (Turbomachines Performance Analysis)

In [1]:
import numpy as np
from scipy.interpolate import CubicSpline

### First approximation

This function makes a first estimation of $k_1$ assuming that $\bar{C_x}$ is the same as in the r.m.s. position. See eqs. (5.22) and (5.27)

In [3]:
def Interpolations():
    r = np.arange(rh,rt+dr,dr)
    ctheta = CubicSpline(rdata,cthetadata)(r)
    f = np.zeros(m)
    for i in range(m-1):
        rm = 0.5*(r[i]+r[i+1])
        cthetam = 0.5*(ctheta[i]+ctheta[i+1])
        drct = r[i+1]*ctheta[i+1]-r[i]*ctheta[i]
        f[i+1] = f[i] + 2*cthetam/rm*drct
    frms = CubicSpline(r,f)(rms)
    k1 = cxm*cxm+frms
    print("First approximation of k1: \n k1 = {}".format(k1))
    return r,f,k1

### More precise computation

This functions iterates to compute more precisely the value of $k_1$. See eqs. (5.28) to (5.30)

In [4]:
def Iterations(r,f,k1):
    Sm = cxm/2*(rt*rt-rh*rh)
    j = 0
    error = 1.0
    while error > 1e-4 :
        dk1 = 0.01*k1
        S1 = 0.5*(rh*np.sqrt(k1-f[1])+rt*np.sqrt(k1-f[m-1]))
        for i in range(1,m):
            S1 += r[i]*np.sqrt(k1-f[i])
        S2 = 0.5*(rh*np.sqrt(k1+dk1-f[1])+rt*np.sqrt(k1+dk1-f[m-1]))
        for i in range(1,m):
            S2 += r[i]*np.sqrt(k1+dk1-f[i])
        S1 = S1*dr
        S2 = S2*dr
        k2 = k1 + dk1*(Sm-S1)/(S2-S1)
        error = np.abs((k2-k1)/k1)
        k1 = 0.5*(k1+k2)
        j += 1
        print("Iteration number {}, k1 = {}".format(j,k1))     
    return k1

#### Printing the results

This function prints de solution of the computation

In [5]:
def Solution(r,f,k1):
    cx = np.zeros(m)
    Q = 0.0
    for i in range(m):
        if k1 > f[i]: cx[i] = np.sqrt(k1-f[i])
        else: cx[i] = 1e-4
        if i > 0: Q += 0.5*(cx[i]+cx[i-1])*np.pi*(r[i]*r[i]-r[i-1]*r[i-1])
    print("{:^15}{:^10}{:^10}".format("Radius","cx","beta2"))
    for i in range(rdata.size):
        #cxans = lagrange(r,cx)(rdata[i])
        cxans = CubicSpline(r,cx)(rdata[i])
        beta2 = np.rad2deg(np.arctan(cthetadata[i]/cxans))
        print("{:^15.4f}{:^10.5f}{:^10.4f}".format(rdata[i],cxans,beta2))
    print("Q = {} m^3/s".format(Q))
    errorQ = np.abs(Qth-Q)/Qth
    print("error = {}".format(errorQ))
    return cx

### Data input

rdata is the set of points where $c_\theta$ is given. cthetadata are the values of $c_\theta$.

In [26]:
#rdata = np.array([0.4,0.5,0.6,0.7,0.8,0.9,1.0])
rdata = np.linspace(0.4,1.0,10)
#cthetadata = rdata            ## forced vortex (solid body swirl)
#cthetadata = np.ones(rdata.size)*1.0   ## constant ctheta
#cthetadata = 1/rdata           ## free-vortex
cthetadata = 0.5*rdata + 0.5/rdata
rh = rdata[0]
rt = rdata[-1]
h = rh/rt
rms = np.sqrt(0.5*(rh*rh+rt*rt))
ctrms = CubicSpline(rdata,cthetadata)(rms)
cxm = 1.0
Qth = cxm*np.pi*(rt*rt-rh*rh)
m = 601   # number of interpolation points
dr = (rt-rh)/(m-1)
print("rms = {}".format(rms))
print("ctheta_rms = {}".format(ctrms))
print("Q = {} m^3/s".format(Qth))

rms = 0.7615773105863909
ctheta_rms = 1.037318176252529
Q = 2.638937829015426 m^3/s


### Computation of $c_x$ and $\beta_2$

In [27]:
r,f,k1 = Interpolations()

First approximation of k1: 
 k1 = 1.8540410963235543


In [28]:
cx = Solution(r,f,k1)

    Radius         cx      beta2   
    0.4000      1.36163   46.8002  
    0.4667      1.29261   45.2681  
    0.5333      1.22638   44.4763  
    0.6000      1.16123   44.3035  
    0.6667      1.09584   44.6711  
    0.7333      1.02903   45.5366  
    0.8000      0.95957   46.8883  
    0.8667      0.88610   48.7458  
    0.9333      0.80689   51.1669  
    1.0000      0.71947   54.2663  
Q = 2.667770119939473 m^3/s
error = 0.010925718145775424


In [29]:
k1 = Iterations(r,f,k1)

Iteration number 1, k1 = 1.8416276824016453
Iteration number 2, k1 = 1.8355193828939171
Iteration number 3, k1 = 1.832497619487235
Iteration number 4, k1 = 1.8309987884387047
Iteration number 5, k1 = 1.8302543675386644
Iteration number 6, k1 = 1.8298843946528827
Iteration number 7, k1 = 1.829700460131939
Iteration number 8, k1 = 1.8296090009677046


In [30]:
cx = Solution(r,f,k1)

    Radius         cx      beta2   
    0.4000      1.35263   46.9898  
    0.4667      1.28312   45.4790  
    0.5333      1.21638   44.7108  
    0.6000      1.15066   44.5653  
    0.6667      1.08464   44.9655  
    0.7333      1.01709   45.8709  
    0.8000      0.94675   47.2725  
    0.8667      0.87221   49.1943  
    0.9333      0.79160   51.7010  
    1.0000      0.70228   54.9204  
Q = 2.6346515989723716 m^3/s
error = 0.0016242254728121297


In [23]:
Q = cxm*np.pi*(rt*rt-rh*rh)
print("Q = {:.4f} m^3/s".format(Q))

Q = 2.6389 m^3/s


In [34]:
r=np.arange(rh,rt+dr,dr)

In [36]:
r.size

601